### Study of the effect of national factors on home prices in the US


**Task:** Using publicly available data on the national factors that impact the supply and demand of homes in the US, build a data science model to study the effect of these variables on home prices.

**Approach:** The following variables are chosen for the study:

1. Unemployment Rate
2. Employment Rate
3. Per capita GDP
4. Median Household Income
5. Construction Prices
6. CPI
7. Interest Rates
8. The number of new houses supplied
9. Working Population
10. Percentage of population above 65
11. Housing subsidies
12. Number of Households

As a proxy for home prices, the S&P **Case-Shiller Index** is used.

**Note:** Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed, and combined to create a dataset using the **Extract Transform Load (ETL)** method. Data for different variables had different frequencies. So, to combine the data, the necessary interpolations are made.


#### Importing neccessary libraries

In [1]:
import numpy as np
import pandas as pd

#### Perform ETL

In [2]:
# Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("/content/CSUSHPISA.csv")

# Changing dtype of date column
df_CS["observation_date"] = pd.to_datetime(df_CS["observation_date"])

# Selecting data till JULY 2023
mask = df_CS["observation_date"] <= "2023-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["observation_date"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["observation_date"]).month
print("Shape of the CASE-SHILLER Index:- ", df_CS.shape)
df_CS.tail()


Shape of the CASE-SHILLER Index:-  (439, 4)


,observation_date,CSUSHPISA,Year,Month
434,2023-03-01,298.958,2023,3
435,2023-04-01,300.345,2023,4
436,2023-05-01,302.411,2023,5
437,2023-06-01,304.400,2023,6
438,2023-07-01,306.598,2023,7


In [3]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("/content/UNRATE.csv")
df_unemp.drop([259], inplace = True)
print("Unemployment Rate Data:- ", df_unemp.shape)
df_unemp.tail()

Unemployment Rate Data:-  (922, 2)


,observation_date,UNRATE
918,2024-07-01,4.3
919,2024-08-01,4.2
920,2024-09-01,4.1
921,2024-10-01,4.1
922,2024-11-01,4.2


In [17]:
# Reading Employment Rate Data into a dataframe
df_emp = pd.read_csv("/content/emprate.csv")
df_emp = df_emp.rename(columns={'LREM64TTUSM156S': 'EmpRate'})
df_emp.drop([259], inplace = True)
print("shape of the Employment Rate Data:- ", df_emp.shape)
df_emp.tail()

shape of the Employment Rate Data:-  (574, 2)


,observation_date,EmpRate
570,2024-07-01,71.78423
571,2024-08-01,71.71506
572,2024-09-01,71.85064
573,2024-10-01,71.68272
574,2024-11-01,71.62031


In [4]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("GDP_per_capita.csv", names = ["observation_date", "A939RX0Q048SBEA"], skiprows = 1)
df_pcgdp = df_pcgdp.rename(columns={'A939RX0Q048SBEA': 'Per_Capita_GDP'})
print("Shape of the Per Capita GDP Data:- ", df_pcgdp.shape)
df_pcgdp.tail()

Shape of the Per Capita GDP Data:-  (311, 2)


,observation_date,Per_Capita_GDP
306,2023-07-01,67916
307,2023-10-01,68351
308,2024-01-01,68549
309,2024-04-01,68977
310,2024-07-01,69399


In [5]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("/content/FEDFUNDS.csv").drop([259])
print("Shape of the Interest rate data:- ",df_Fed_rate.shape)
df_Fed_rate.tail()

Shape of the Interest rate data:-  (845, 2)


,observation_date,FEDFUNDS
841,2024-08-01,5.33
842,2024-09-01,5.13
843,2024-10-01,4.83
844,2024-11-01,4.64
845,2024-12-01,4.48


In [7]:
# Reading Construction Material Data into a dataframe
df_cons_price_index = pd.read_csv("/content/construction_price_ppi.csv", names = ["observation_date", "WPUSI012011"], skiprows = 1)
df_cons_price_index = df_cons_price_index.rename(columns={'WPUSI012011': 'Cons_Material'})
df_cons_price_index.drop([259], inplace = True)
print("Shape of the Construction Material Data:- ", df_cons_price_index.shape)
df_cons_price_index.tail()

Shape of the Construction Material Data:-  (934, 2)


,observation_date,Cons_Material
930,2024-07-01,325.556
931,2024-08-01,324.626
932,2024-09-01,324.032
933,2024-10-01,325.380
934,2024-11-01,327.047


In [8]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["observation_date", "CPIAUCSL"], skiprows = 1).drop([259])
df_CPI = df_CPI.rename(columns={'CPIAUCSL': 'CPI'})
print("Shape of the Consumer Price Index:- ", df_CPI.shape)
df_CPI.tail()

Shape of the Consumer Price Index:-  (934, 2)


,observation_date,CPI
930,2024-07-01,313.534
931,2024-08-01,314.121
932,2024-09-01,314.686
933,2024-10-01,315.454
934,2024-11-01,316.441


In [9]:
# Monthly new house supply
df_house = pd.read_csv("/content/MSACSR.csv", names = ["observation_date", "MSACSR"], skiprows = 1).drop([259])
df_house = df_house.rename(columns={'MSACSR': 'Houses'})
print("Shape of the monthly house supply data:- ", df_house.shape)
df_house.tail()


Shape of the monthly house supply data:-  (742, 2)


,observation_date,Houses
738,2024-07-01,7.9
739,2024-08-01,8.2
740,2024-09-01,7.7
741,2024-10-01,9.2
742,2024-11-01,8.9


In [10]:
# Population above 65

df_oldpop = pd.read_csv("/content/old_age_pop.csv", names = ["observation_date", "old_age_pop"], skiprows = 1)
df_oldpop['observation_date'] = pd.to_datetime(df_oldpop['observation_date'], format="%Y-%m-%d").dt.strftime("%Y-%m-%d")
print("Shape of the population data age above 65:- ", df_oldpop.shape)
df_oldpop.tail()

Shape of the population data age above 65:-  (64, 2)


,observation_date,old_age_pop
59,2019-01-01,15.672885
60,2020-01-01,16.071912
61,2021-01-01,16.473770
62,2022-01-01,16.919620
63,2023-01-01,17.431819


In [ ]:
# Urban Population Percent

df_urban = pd.read_csv("/content/POPTHM.csv")
df_urban['DATE'] = pd.to_datetime(df_urban['DATE'], format="%d-%m-%Y").dt.strftime("%Y-%m-%d")
print("Shape of the urban population percent data:- ", df_urban.shape)
df_urban.tail()

Shape of the urban population percent data:-  (21, 2)


,DATE,urban_pop_us
16,2018-01-01,82.3
17,2019-01-01,82.5
18,2020-01-01,82.7
19,2021-01-01,82.9
20,2022-01-01,83.1


In [11]:
# Housing Subsidies

df_subsidy = pd.read_csv("/content/housing_subsidies.csv", names = ["observation_date", "Subsidy"], skiprows = 1)
print("Shape of the housing subsidies:- ", df_subsidy.shape)
df_subsidy.tail()


Shape of the housing subsidies:-  (64, 2)


,observation_date,Subsidy
59,2019-01-01,40.185
60,2020-01-01,44.147
61,2021-01-01,45.299
62,2022-01-01,48.021
63,2023-01-01,53.573


In [12]:
# Working age population

df_working = pd.read_csv("/content/working_age_population.csv", names = ["observation_date", "LFWA64TTUSM647S"], skiprows = 1).drop([259])
df_working = df_working.rename(columns={'LFWA64TTUSM647S': 'working_age_pop'})
print("Shape of the working age population:- ", df_working.shape)
df_working.tail()

Shape of the working age population:-  (574, 2)


,observation_date,working_age_pop
570,2024-07-01,208875500
571,2024-08-01,208926800
572,2024-09-01,209079300
573,2024-10-01,209076300
574,2024-11-01,209238600


In [13]:
# Real Median Household Income

df_income = pd.read_csv("/content/median_household_income.csv", names = ["observation_date", "MEHOINUSA672N"], skiprows = 1)
df_income = df_income.rename(columns={'MEHOINUSA672N': 'median_income'})
print("Shape of the median household income data:- ", df_income.shape)
df_income.tail()


Shape of the median household income data:-  (40, 2)


,observation_date,median_income
35,2019-01-01,81210
36,2020-01-01,79560
37,2021-01-01,79260
38,2022-01-01,77540
39,2023-01-01,80610


In [14]:
# Total number of households

df_households = pd.read_csv("/content/household.csv", names = ["observation_date", "TTLHH"], skiprows = 1)
df_households = df_households.rename(columns={'TTLHH': 'Num_Households'})
print("Shape of the total households data:- ", df_households.shape)
df_households.tail()


Shape of the total households data:-  (85, 2)


,observation_date,Num_Households
80,2020-01-01,128451.0
81,2021-01-01,129224.0
82,2022-01-01,131202.0
83,2023-01-01,131434.0
84,2024-01-01,132216.0


In [18]:
# Merging Per Capita GDP
df_pcgdp["observation_date"] = pd.to_datetime(df_pcgdp["observation_date"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")
df_CS.head()


,observation_date,CSUSHPISA,Year,Month,Per_Capita_GDP
0,1987-01-01,63.963,1987,1,37132.0
1,1987-02-01,64.422,1987,2,NaN
2,1987-03-01,64.734,1987,3,NaN
3,1987-04-01,65.130,1987,4,37454.0
4,1987-05-01,65.562,1987,5,NaN


In [19]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_CS, df_working, df_house, df_CPI, df_unemp, df_emp, df_cons_price_index, df_Fed_rate]
for df1 in df_bymonth:
    df1["observation_date"] = pd.to_datetime(df1["observation_date"])
    df1 = df1.set_index("observation_date")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()


(936, 11)


,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS
observation_date,,,,,,,,,,,
1947-01-01,NaN,NaN,NaN,NaN,NaN,NaN,21.48,NaN,NaN,22.2,NaN
1947-02-01,NaN,NaN,NaN,NaN,NaN,NaN,21.62,NaN,NaN,22.5,NaN
1947-03-01,NaN,NaN,NaN,NaN,NaN,NaN,22.00,NaN,NaN,22.9,NaN
1947-04-01,NaN,NaN,NaN,NaN,NaN,NaN,22.00,NaN,NaN,23.2,NaN
1947-05-01,NaN,NaN,NaN,NaN,NaN,NaN,21.95,NaN,NaN,23.3,NaN


In [20]:
# Merging other dataframes
others = [df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["observation_date"]).year
        df1.set_index("observation_date", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("observation_date", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["observation_date"] = df_CS["observation_date"]
df.set_index("observation_date", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy,old_age_pop
observation_date,,,,,,,,,,,,,,,
1987-01-01,NaN,NaN,NaN,NaN,NaN,NaN,21.48,NaN,NaN,22.2,NaN,NaN,NaN,NaN,NaN
1987-02-01,NaN,NaN,NaN,NaN,NaN,NaN,21.62,NaN,NaN,22.5,NaN,NaN,NaN,NaN,NaN
1987-03-01,NaN,NaN,NaN,NaN,NaN,NaN,22.00,NaN,NaN,22.9,NaN,NaN,NaN,NaN,NaN
1987-04-01,NaN,NaN,NaN,NaN,NaN,NaN,22.00,NaN,NaN,23.2,NaN,NaN,NaN,NaN,NaN
1987-05-01,NaN,NaN,NaN,NaN,NaN,NaN,21.95,NaN,NaN,23.3,NaN,NaN,NaN,NaN,NaN


In [21]:
print(df.shape)

(936, 15)


Check missing values (NAN)

In [22]:
df.isna().sum()

,0
CSUSHPISA,497
Year,497
Month,497
Per_Capita_GDP,789
working_age_pop,362
Houses,194
CPI,2
UNRATE,14
EmpRate,362
Cons_Material,2


Theres are alot of missing values so to handle them for particular feature. I used different technique which is mentioned below.



In [24]:
# Handling missing values in the dataset

# 1. Drop rows with missing values in critical columns
df = df.dropna(subset=["CSUSHPISA", "Year", "Month", "Num_Households", "median_income", "Subsidy", "old_age_pop"])

# 2. Fill missing values in the "Per_Capita_GDP" column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate(method="linear")

# 3. Fill missing values in the "working_age_pop" column using linear interpolation
df["working_age_pop"] = df["working_age_pop"].interpolate(method="linear")

# 4. Fill missing values in the "Houses" column using forward fill
df["Houses"] = df["Houses"].fillna(method="ffill")

# 5. Fill missing values in the "CPI" column using the median value
df["CPI"] = df["CPI"].fillna(df["CPI"].median())

# 6. Fill missing values in the "UNRATE" column using the mean value
df["UNRATE"] = df["UNRATE"].fillna(df["UNRATE"].mean())

# 7. Fill missing values in the "EmpRate" column using linear interpolation
df["EmpRate"] = df["EmpRate"].interpolate(method="linear")

# 8. Fill missing values in the "Cons_Material" column using the median value
df["Cons_Material"] = df["Cons_Material"].fillna(df["Cons_Material"].median())

# 9. Fill missing values in the "FEDFUNDS" column using linear interpolation
df["FEDFUNDS"] = df["FEDFUNDS"].interpolate(method="linear")



<ipython-input-24-d4dade52d60c>:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df["Houses"] = df["Houses"].fillna(method="ffill")


In [38]:
# Final check for remaining missing values
print(df.isna().sum())

# Final dataset shape
print(df.shape)


CSUSHPISA          0
Year               0
Month              0
Per_Capita_GDP     0
working_age_pop    0
Houses             0
CPI                0
UNRATE             0
EmpRate            0
Cons_Material      0
FEDFUNDS           0
Num_Households     0
median_income      0
Subsidy            0
old_age_pop        0
dtype: int64
(439, 15)


In [35]:
if "Year" in df.columns and "Month" in df.columns:
    # Convert Year and Month to integers, then form a valid date
    df["observation_date"] = pd.to_datetime(
        df["Year"].astype(int).astype(str) + "-" + df["Month"].astype(int).astype(str).str.zfill(2) + "-01"
    )
else:
    # Fallback: Generate a sequential date range
    df["observation_date"] = pd.date_range(start="2002-01-01", periods=len(df), freq="MS")

# Set observation_date as the index
df = df.set_index("observation_date")

# Verify the result
print(df.index)




DatetimeIndex(['1987-01-01', '1987-02-01', '1987-03-01', '1987-04-01',
               '1987-05-01', '1987-06-01', '1987-07-01', '1987-08-01',
               '1987-09-01', '1987-10-01',
               ...
               '2022-10-01', '2022-11-01', '2022-12-01', '2023-01-01',
               '2023-02-01', '2023-03-01', '2023-04-01', '2023-05-01',
               '2023-06-01', '2023-07-01'],
              dtype='datetime64[ns]', name='observation_date', length=439, freq=None)


In [36]:
df

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy,old_age_pop
observation_date,,,,,,,,,,,,,,,
1987-01-01,63.963,1987.0,1.0,37132.000000,153840000.0,6.0,111.400,6.6,70.15342,107.600,6.43,89479.0,63060.0,11.506,11.990180
1987-02-01,64.422,1987.0,2.0,37239.333333,154053700.0,6.2,111.800,6.6,70.28307,107.900,6.10,89479.0,63060.0,11.506,11.990180
1987-03-01,64.734,1987.0,3.0,37346.666667,154196500.0,6.0,112.200,6.6,70.31552,108.100,6.13,89479.0,63060.0,11.506,11.990180
1987-04-01,65.130,1987.0,4.0,37454.000000,154342400.0,6.0,112.700,6.3,70.50945,108.300,6.37,89479.0,63060.0,11.506,11.990180
1987-05-01,65.562,1987.0,5.0,37531.000000,154526500.0,6.7,113.000,6.3,70.81400,108.300,6.85,89479.0,63060.0,11.506,11.990180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-01,298.958,2023.0,3.0,67185.666667,208204600.0,8.1,301.744,3.5,71.85798,331.729,4.65,131434.0,80610.0,53.573,17.431819
2023-04-01,300.345,2023.0,4.0,67295.000000,208312600.0,7.5,303.032,3.4,71.95400,333.366,4.83,131434.0,80610.0,53.573,17.431819
2023-05-01,302.411,2023.0,5.0,67502.000000,208514000.0,6.9,303.365,3.7,71.93533,337.473,5.06,131434.0,80610.0,53.573,17.431819


In [37]:
df.dropna(inplace = True)

In [39]:
print("Shape of the dataframe after preprocessing:- ", df.shape)

Shape of the dataframe after preprocessing:-  (439, 15)


This is our preprocessed datset. Let's save it as "prepared_data.csv".


In [40]:
df.to_csv("prepared_data.csv")

In [41]:
us_house_price_df = pd.read_csv("prepared_data.csv").set_index("observation_date")
us_house_price_df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,working_age_pop,Houses,CPI,UNRATE,EmpRate,Cons_Material,FEDFUNDS,Num_Households,median_income,Subsidy,old_age_pop
observation_date,,,,,,,,,,,,,,,
1987-01-01,63.963,1987.0,1.0,37132.000000,153840000.0,6.0,111.4,6.6,70.15342,107.6,6.43,89479.0,63060.0,11.506,11.99018
1987-02-01,64.422,1987.0,2.0,37239.333333,154053700.0,6.2,111.8,6.6,70.28307,107.9,6.10,89479.0,63060.0,11.506,11.99018
1987-03-01,64.734,1987.0,3.0,37346.666667,154196500.0,6.0,112.2,6.6,70.31552,108.1,6.13,89479.0,63060.0,11.506,11.99018
1987-04-01,65.130,1987.0,4.0,37454.000000,154342400.0,6.0,112.7,6.3,70.50945,108.3,6.37,89479.0,63060.0,11.506,11.99018
1987-05-01,65.562,1987.0,5.0,37531.000000,154526500.0,6.7,113.0,6.3,70.81400,108.3,6.85,89479.0,63060.0,11.506,11.99018
